# Making Python Code Faster with Numba

In [23]:
from numba import jit
import random

def monte_carlo_pi(nSamples):
    acc=0
    for i in range(nSamples):
        x = random.random()
        y = random.random()
        if (x**2 + y**2) < 1.0:
            acc += 1
    return 4.0 * acc/nSamples

In [24]:
%time monte_carlo_pi(10000)

CPU times: total: 0 ns
Wall time: 3.06 ms


3.1212

In [25]:
monte_carlo_pi_jit = jit()(monte_carlo_pi)

C:\Users\anura\AppData\Local\Temp\ipykernel_24508\3627295421.py:1: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  monte_carlo_pi_jit = jit()(monte_carlo_pi)


In [26]:
%time monte_carlo_pi_jit(10000)

CPU times: total: 0 ns
Wall time: 96.4 ms


3.1388

Time increases if you run it once with jit because of compilation time, but if you run it again it decreases.

## Another Example

In [27]:
from numba import jit, njit, vectorize

In [28]:
def original_func(input_list):
    output_list = []
    for item in input_list:
        if item%2 == 0:
            output_list.append(2)
        else:
            output_list.append('1')
    return output_list

test_array = list(range(100000))

In [29]:
%time _ = original_func(test_array)

CPU times: total: 0 ns
Wall time: 6.13 ms


In [30]:
jitted_func = jit()(original_func)

C:\Users\anura\AppData\Local\Temp\ipykernel_24508\868546817.py:1: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  jitted_func = jit()(original_func)


In [31]:
%time _ = jitted_func(test_array)

C:\Users\anura\AppData\Local\Temp\ipykernel_24508\2518354451.py:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "original_func" failed type inference due to: Invalid use of BoundFunction(list.append for list(int64)<iv=None>) with parameters (Literal[str](1))

During: resolving callee type: BoundFunction(list.append for list(int64)<iv=None>)
During: typing of call at C:\Users\anura\AppData\Local\Temp\ipykernel_24508\2518354451.py (7)


File "..\..\..\..\AppData\Local\Temp\ipykernel_24508\2518354451.py", line 7:
<source missing, REPL/exec in use?>

  def original_func(input_list):
C:\Users\anura\AppData\Local\Temp\ipykernel_24508\2518354451.py:1: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "original_func" failed type inference due to: Cannot determine Numba type of <class 'numba.core.dispatcher.LiftedLoop'>

File "..\..\..\..\AppData\Local\Temp\ipykernel_24508\2518354451.py",

CPU times: total: 15.6 ms
Wall time: 291 ms


Numba cannot tell whether to append number or a string.
```python
if item%2 == 0:
    output_list.append(2)
else:
    output_list.append('1')
```
```
Compilation is falling back to object mode WITH looplifting enabled because Function "original_func" failed type inference due to: Invalid use of BoundFunction(list.append for list(int64)<iv=None>) with parameters (Literal[str](1))

During: resolving callee type: BoundFunction(list.append for list(int64)<iv=None>)
```

In [32]:
%time _ = jitted_func(test_array)

CPU times: total: 0 ns
Wall time: 14.8 ms


In [33]:
jitted_func = jit(nopython=True)(original_func)

In [34]:
%time _ = jitted_func(test_array)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1mInvalid use of BoundFunction(list.append for list(int64)<iv=None>) with parameters (Literal[str](1))
[0m
[0m[1mDuring: resolving callee type: BoundFunction(list.append for list(int64)<iv=None>)[0m
[0m[1mDuring: typing of call at C:\Users\anura\AppData\Local\Temp\ipykernel_24508\2518354451.py (7)
[0m
[1m
File "..\..\..\..\AppData\Local\Temp\ipykernel_24508\2518354451.py", line 7:[0m
[1m<source missing, REPL/exec in use?>[0m


So now with ```nopython=True``` we get the error: ```TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Invalid use of BoundFunction(list.append for list(int64)) with parameters (Literal[str](1))```